In [4]:
# turnquest_advanced.py
"""
TurnQuest Advanced - Pygame Turn-Based Battle with:
- Bigger window (1400x800)
- Multiple status effects (Burn, Poison, Stun, Defense Down)
- Move upgrades at levels 3, 5, 7+
- Death counter
"""

import pygame, sys, random
from dataclasses import dataclass, field
from typing import List, Optional, Callable, Dict

# -------------------- Config --------------------
WIDTH, HEIGHT = 1600, 900
FPS = 60
FONT_NAME = None
BG_COLOR = (18, 20, 28)
PANEL = (28, 34, 45)
ACCENT = (99, 155, 255)
TEXT = (230, 230, 240)
HP_GREEN = (94, 221, 121)
HP_RED = (235, 92, 92)
LOG_BG = (14, 16, 20)
BUTTON = (48, 58, 75)
BUTTON_HOVER = (70, 95, 140)
DISABLED = (70, 70, 80)
FADE_COLOR = (0, 0, 0, 160)

pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("TurnQuest Advanced")
clock = pygame.time.Clock()
font = pygame.font.Font(FONT_NAME, 18)
bigfont = pygame.font.Font(FONT_NAME, 28)
title_font = pygame.font.Font(FONT_NAME, 46)

# -------------------- Data Models --------------------
@dataclass
class Status:
    name: str
    duration: int
    potency: int = 0  # damage or buff amount

@dataclass
class Move:
    name: str
    desc: str
    power: int = 0
    accuracy: float = 1.0
    cooldown: int = 0
    current_cd: int = 0
    target_self: bool = False
    apply_status: Optional[Callable[['Character','Character'], Optional[Status]]] = None

    def ready(self):
        return self.current_cd <= 0

    def trigger_cd(self):
        if self.cooldown > 0:
            self.current_cd = self.cooldown

    def tick_cd(self):
        if self.current_cd > 0:
            self.current_cd -= 1

@dataclass
class Item:
    name: str
    desc: str
    use: Callable[['Character'], str]
    qty: int = 1

@dataclass
class Character:
    name: str
    level: int
    max_hp: int
    hp: int
    atk: int
    defense: int
    speed: int
    moves: List[Move]
    statuses: List[Status] = field(default_factory=list)
    buffs: Dict[str, Status] = field(default_factory=dict)
    xp: int = 0

    def is_alive(self):
        return self.hp > 0

    def take_damage(self, amt: int):
        amt = max(0, amt)
        self.hp = max(0, min(self.max_hp, self.hp - amt))
        return amt

    def heal(self, amt: int):
        if amt <= 0:
            return 0
        healed = max(0, min(self.max_hp - self.hp, amt))
        self.hp += healed
        return healed

    def add_status(self, s: Status):
        for ex in self.statuses:
            if ex.name == s.name:
                ex.duration = s.duration
                ex.potency = s.potency
                return
        self.statuses.append(s)

    def has_status(self, name: str):
        return any(s.name == name for s in self.statuses)

    def tick_statuses(self):
        msgs = []
        for s in self.statuses[:]:
            if s.name == 'Burn' or s.name == 'Poison':
                dmg = s.potency
                self.take_damage(dmg)
                msgs.append(f"{self.name} takes {dmg} {s.name.lower()} damage.")
            elif s.name == 'Stun':
                msgs.append(f"{self.name} is stunned and skips turn!")
            elif s.name == 'Defense Down':
                msgs.append(f"{self.name}'s defense is lowered!")
            s.duration -= 1
            if s.duration <= 0:
                self.statuses.remove(s)
                msgs.append(f"{self.name} is no longer {s.name.lower()}.")
        # tick buffs
        for k, b in list(self.buffs.items()):
            b.duration -= 1
            if b.duration <= 0:
                del self.buffs[k]
                msgs.append(f"{self.name}'s {k} buff wore off.")
        return msgs

    def get_stat(self, stat: str):
        base = getattr(self, stat)
        buff = self.buffs.get(stat)
        if buff:
            return base + buff.potency
        return base

    def add_buff(self, stat: str, amount: int, duration: int):
        self.buffs[stat] = Status(name=f"buff:{stat}", duration=duration, potency=amount)

    def tick_move_cds(self):
        for m in self.moves:
            m.tick_cd()

# -------------------- Game Data --------------------
def make_player_moves():
    return [
        Move("Slash", "A solid physical strike.", power=18, accuracy=0.95, cooldown=0),
        Move("Power Strike", "Heavy hit, lower accuracy.", power=30, accuracy=0.75, cooldown=2),
        Move("First Aid", "Heal yourself for moderate HP.", power=-25, accuracy=1.0, cooldown=3, target_self=True),
        Move("Flame Shot", "Deal fire damage and may burn.", power=15, accuracy=0.9, cooldown=3,
             apply_status=lambda u, t: Status("Burn", duration=3, potency=5))
    ]

def make_enemy_moves(level):
    return [
        Move("Bite", "Enemy bite.", power=12 + level*2, accuracy=0.95),
        Move("Rage", "Powerful blow.", power=20 + level*3, accuracy=0.8, cooldown=2),
        Move("Spit", "May burn target.", power=8 + level, accuracy=0.85,
             apply_status=lambda u, t: Status("Burn", duration=3, potency=4)),
        Move("Poison Fang", "May poison.", power=6+level, accuracy=0.9,
             apply_status=lambda u,t: Status("Poison", duration=3, potency=3))
    ]

def make_items():
    return {
        'potion': Item("Potion", "Heals 40 HP.", qty=3, use=lambda c: f"{c.name} healed {c.heal(40)} HP."),
        'antidote': Item("Antidote", "Removes Burn/Poison.", qty=1, use=lambda c: _use_antidote(c))
    }

def _use_antidote(c: Character):
    removed = False
    for s in c.statuses[:]:
        if s.name in ['Burn','Poison']:
            c.statuses.remove(s)
            removed = True
    return f"{c.name} had their status removed." if removed else f"{c.name} had no status to remove."

def calculate_damage(attacker: Character, defender: Character, base_power: int):
    atk = attacker.get_stat('atk')
    defense = max(1, defender.get_stat('defense'))
    raw = (atk * base_power) / defense
    variation = random.uniform(0.85, 1.15)
    dmg = int(max(1, raw * variation))
    return dmg

def attempt_hit(move: Move):
    return random.random() <= move.accuracy

def enemy_choose_move(enemy: Character, player: Character):
    usable = [m for m in enemy.moves if m.ready()]
    if not usable: return None
    if enemy.hp < enemy.max_hp*0.35:
        candidates = sorted(usable, key=lambda mm: mm.power)
        return candidates[0]
    candidates = sorted(usable, key=lambda mm: mm.power, reverse=True)
    if random.random()<0.2:
        for m in usable:
            if m.apply_status:
                return m
    return candidates[0]

def generate_enemy(player_level: int) -> Character:
    lvl = max(1, player_level + random.choice([-1,0,1]))
    base_hp = 60 + lvl*10
    enemy = Character(
        name=random.choice(['Goblin','Wolf','Bandit','Slime','Rogue']),
        level=lvl,
        max_hp=base_hp,
        hp=base_hp,
        atk=8+lvl*2,
        defense=4+lvl,
        speed=5+lvl,
        moves=make_enemy_moves(lvl)
    )
    return enemy

# -------------------- UI Widget --------------------
class Button:
    def __init__(self, rect, text, action=None, font=font, enabled=True):
        self.rect = pygame.Rect(rect)
        self.text = text
        self.action = action
        self.font = font
        self.enabled = enabled
        self.hover = False
    def draw(self, surf):
        col = BUTTON
        if not self.enabled: col=DISABLED
        elif self.hover: col=BUTTON_HOVER
        pygame.draw.rect(surf,col,self.rect,border_radius=8)
        txt = self.font.render(self.text,True,TEXT)
        txt_r = txt.get_rect(center=self.rect.center)
        surf.blit(txt,txt_r)
    def handle_event(self, ev):
        if ev.type == pygame.MOUSEMOTION:
            self.hover = self.rect.collidepoint(ev.pos)
        elif ev.type == pygame.MOUSEBUTTONDOWN and ev.button==1:
            if self.enabled and self.rect.collidepoint(ev.pos):
                if self.action: self.action()

# -------------------- Game Class --------------------
class TurnQuestGame:
    def __init__(self):
        self.state = 'start'
        self.player_name = "Hero"
        self.player = None
        self.enemy = None
        self.inventory = None
        self.log:List[str]=[]
        self.encounters=0
        self.selected_move=None
        self.message_timer=0.0
        self.anim_t=0.0
        self.death_count = 0
        self.ui_buttons:List[Button]=[]
        self.move_buttons:List[Button]=[]
        self.item_buttons:List[Button]=[]
        self.quit_button=None
        self.continue_button=None
        self.rest_button=None
        self.setup_start_screen()

    def setup_start_screen(self):
        self.input_active=False
        self.input_text=""
        self.name_box=pygame.Rect(WIDTH//2-250, HEIGHT//2-30, 500, 48)
        self.start_btn=Button((WIDTH//2-120, HEIGHT//2+40, 120, 48),"Start",action=self.start_game)
        self.exit_btn=Button((WIDTH//2+10, HEIGHT//2+40, 120, 48),"Quit",action=self.quit_game)

    def start_game(self):
        name=self.input_text.strip() or "Hero"
        self.player_name=name
        self.player=Character(name=self.player_name,level=1,max_hp=120,hp=120,atk=14,defense=9,speed=10,moves=make_player_moves())
        self.inventory=make_items()
        self.encounters=0
        self.log=[]
        self.next_encounter()
        self.state='battle'

    def quit_game(self):
        pygame.quit()
        sys.exit(0)

    def next_encounter(self):
        self.encounters+=1
        self.enemy=generate_enemy(self.player.level)
        self.push_log(f"A wild {self.enemy.name} (Lv {self.enemy.level}) appears!")
        self.state='battle'
        self.create_battle_ui()
        self.tick_start_of_round()

    # --- UI & Button layout ---
    def create_battle_ui(self):
        self.ui_buttons.clear()
        self.move_buttons.clear()
        self.item_buttons.clear()
        # Moves (left)
        left=28
        base_y=HEIGHT-300
        btn_w=280
        btn_h=50
        for i,mv in enumerate(self.player.moves):
            rect=(left, base_y + i*(btn_h+8), btn_w, btn_h)
            b=Button(rect,f"{mv.name} {'(CD:'+str(mv.current_cd)+')' if mv.current_cd>0 else ''}",action=lambda m=mv:self.on_move_click(m))
            self.move_buttons.append(b)
            self.ui_buttons.append(b)
        # Items (right)
        x2=left+btn_w+18
        for j,key in enumerate(self.inventory.keys()):
            it=self.inventory[key]
            rect=(x2, base_y+j*(btn_h+8), 220, btn_h)
            b=Button(rect,f"{it.name} x{it.qty}",action=lambda k=key:self.on_item_click(k))
            self.item_buttons.append(b)
            self.ui_buttons.append(b)
        # Pass
        pass_rect=(x2+240, base_y, 140, btn_h)
        self.ui_buttons.append(Button(pass_rect,"Pass",action=self.player_pass))
        # Quit
        self.quit_button=Button((WIDTH-120,18,100,40),"Quit",action=self.quit_game)
        self.ui_buttons.append(self.quit_button)
        self.refresh_button_states()

    def refresh_button_states(self):
        for b,m in zip(self.move_buttons,self.player.moves):
            cd=f"(CD:{m.current_cd})" if m.current_cd>0 else ""
            b.text=f"{m.name} {cd}"
            b.enabled=m.ready() and self.player.is_alive() and self.enemy.is_alive() and self.state=='battle'
        for i,key in enumerate(self.inventory.keys()):
            it=self.inventory[key]
            if i<len(self.item_buttons):
                self.item_buttons[i].text=f"{it.name} x{it.qty}"
                self.item_buttons[i].enabled=(it.qty>0 and self.state=='battle')
        if self.quit_button:
            self.quit_button.enabled=True

    def push_log(self,txt:str):
        self.log.insert(0,txt)
        if len(self.log)>10: self.log=self.log[:10]

    # --- Player interactions ---
    def on_move_click(self,move:Move):
        if not move.ready():
            self.push_log(f"{move.name} is on cooldown ({move.current_cd} turns).")
            return
        if self.message_timer>0: return
        self.resolve_player_move(move)

    def on_item_click(self,key:str):
        if self.message_timer>0: return
        it=self.inventory[key]
        if it.qty<=0:
            self.push_log("No item left.")
            return
        res=it.use(self.player)
        it.qty-=1
        self.push_log(res)
        self.end_player_action()

    def player_pass(self):
        if self.message_timer>0: return
        self.push_log(f"{self.player.name} passes the turn.")
        self.end_player_action()

    # --- Move resolution ---
    def resolve_player_move(self,move:Move):
        self.push_log(f"{self.player.name} uses {move.name}!")
        if not attempt_hit(move):
            self.push_log("But it missed!")
            move.trigger_cd()
            self.refresh_button_states()
            self.message_timer=0.7
            return
        if move.power<0:
            healed=-move.power
            amt=self.player.heal(healed)
            self.push_log(f"{self.player.name} heals {amt} HP.")
        else:
            dmg=calculate_damage(self.player,self.enemy,move.power)
            inflicted=self.enemy.take_damage(dmg)
            self.push_log(f"It deals {inflicted} damage to {self.enemy.name} ({self.enemy.hp}/{self.enemy.max_hp}).")
        if move.apply_status:
            s=move.apply_status(self.player,self.enemy)
            if s:
                self.enemy.add_status(s)
                self.push_log(f"{self.enemy.name} afflicted with {s.name} ({s.duration}).")
        move.trigger_cd()
        self.refresh_button_states()
        if not self.enemy.is_alive():
            self.push_log(f"{self.enemy.name} was defeated!")
            self.message_timer=0.6
            self.message_timer_action=self.handle_victory
            return
        self.message_timer=0.8
        self.message_timer_action=self.enemy_turn

    def enemy_turn(self):
        if not self.enemy.is_alive() or not self.player.is_alive(): return
        # check stun
        if self.enemy.has_status("Stun"):
            self.push_log(f"{self.enemy.name} is stunned and cannot act!")
            # remove stun
            self.enemy.statuses=[s for s in self.enemy.statuses if s.name!="Stun"]
            self.message_timer=0.6
            self.message_timer_action=self.tick_start_of_round
            return
        move=enemy_choose_move(self.enemy,self.player)
        if not move:
            self.push_log(f"{self.enemy.name} cannot act.")
            self.message_timer=0.6
            self.message_timer_action=self.tick_start_of_round
            return
        self.push_log(f"{self.enemy.name} uses {move.name}!")
        if not attempt_hit(move):
            self.push_log("It missed!")
            move.trigger_cd()
            self.refresh_button_states()
            self.message_timer=0.6
            self.message_timer_action=self.tick_start_of_round
            return
        if move.power<0:
            amt=self.enemy.heal(-move.power)
            self.push_log(f"{self.enemy.name} heals {amt} HP.")
        else:
            dmg=calculate_damage(self.enemy,self.player,move.power)
            inflicted=self.player.take_damage(dmg)
            self.push_log(f"{self.enemy.name} deals {inflicted} damage to {self.player.name} ({self.player.hp}/{self.player.max_hp}).")
        if move.apply_status:
            s=move.apply_status(self.enemy,self.player)
            if s:
                self.player.add_status(s)
                self.push_log(f"{self.player.name} afflicted with {s.name} ({s.duration}).")
        move.trigger_cd()
        self.refresh_button_states()
        if not self.player.is_alive():
            self.push_log(f"{self.player.name} was defeated...")
            self.message_timer=1.2
            self.message_timer_action=self.handle_defeat
            return
        self.message_timer=0.6
        self.message_timer_action=self.tick_start_of_round

    def end_player_action(self):
        self.message_timer=0.6
        self.message_timer_action=self.enemy_turn

    # --- Round management ---
    def tick_start_of_round(self):
        # tick statuses
        for msg in self.player.tick_statuses(): self.push_log(msg)
        for msg in self.enemy.tick_statuses(): self.push_log(msg)
        # tick cooldowns
        self.player.tick_move_cds()
        self.enemy.tick_move_cds()
        self.refresh_button_states()

    def handle_victory(self):
        self.player.xp += self.enemy.level*15
        self.push_log(f"{self.player.name} gains {self.enemy.level*15} XP!")
        self.check_level_up()
        self.next_encounter()

    def handle_defeat(self):
        self.death_count+=1
        self.player.hp=self.player.max_hp
        self.player.level=max(1,self.player.level-1)
        self.player.moves=make_player_moves()
        self.push_log(f"{self.player.name} was revived at full HP. Deaths: {self.death_count}")
        self.next_encounter()

    # --- Level Up ---
    def check_level_up(self):
        leveled=False
        xp_needed=50*self.player.level
        while self.player.xp>=xp_needed:
            self.player.xp-=xp_needed
            self.player.level+=1
            self.player.max_hp+=12
            self.player.hp=self.player.max_hp
            self.player.atk+=3
            self.player.defense+=2
            self.player.speed+=1
            leveled=True
            self.push_log(f"{self.player.name} leveled up to {self.player.level}!")
            self.try_upgrade_move()
            xp_needed=50*self.player.level
        return leveled

    def try_upgrade_move(self):
        # Only certain levels
        if self.player.level in [3,5,7,10]:
            mv=random.choice(self.player.moves)
            mv.power+=5
            self.push_log(f"{mv.name} grew stronger! Power is now {mv.power}.")

    # --- Main Loop ---
    def handle_events(self,ev):
        if self.state=='start':
            if ev.type==pygame.KEYDOWN:
                if self.input_active:
                    if ev.key==pygame.K_RETURN:
                        self.start_game()
                    elif ev.key==pygame.K_BACKSPACE:
                        self.input_text=self.input_text[:-1]
                    else:
                        self.input_text+=ev.unicode
            self.start_btn.handle_event(ev)
            self.exit_btn.handle_event(ev)
        elif self.state=='battle':
            for b in self.ui_buttons:
                b.handle_event(ev)

    def update(self,dt):
        if self.message_timer>0:
            self.message_timer-=dt
            if self.message_timer<=0 and hasattr(self,'message_timer_action'):
                self.message_timer_action()
                del self.message_timer_action

    def draw(self,surf):
        surf.fill(BG_COLOR)
        if self.state=='start':
            txt=title_font.render("TurnQuest Advanced",True,ACCENT)
            surf.blit(txt,txt.get_rect(center=(WIDTH//2, HEIGHT//4)))
            pygame.draw.rect(surf,PANEL,self.name_box,border_radius=8)
            txt2=font.render(self.input_text or "Enter hero name",True,TEXT)
            surf.blit(txt2,(self.name_box.x+8,self.name_box.y+10))
            self.start_btn.draw(surf)
            self.exit_btn.draw(surf)
        elif self.state=='battle':
            # Panels
            pygame.draw.rect(surf,PANEL,(28,28,500,180),border_radius=10)
            pygame.draw.rect(surf,PANEL,(WIDTH-528,28,500,180),border_radius=10)
            # Player info
            ph=self.player.hp/self.player.max_hp
            pygame.draw.rect(surf,HP_RED,(50,120,400,28),border_radius=6)
            pygame.draw.rect(surf,HP_GREEN,(50,120,int(400*ph),28),border_radius=6)
            txt=font.render(f"{self.player.name} Lv {self.player.level} HP:{self.player.hp}/{self.player.max_hp}",True,TEXT)
            surf.blit(txt,(50,90))
            # Enemy info
            eh=self.enemy.hp/self.enemy.max_hp
            pygame.draw.rect(surf,HP_RED,(WIDTH-500,120,400,28),border_radius=6)
            pygame.draw.rect(surf,HP_GREEN,(WIDTH-500,120,int(400*eh),28),border_radius=6)
            txt=font.render(f"{self.enemy.name} Lv {self.enemy.level} HP:{self.enemy.hp}/{self.enemy.max_hp}",True,TEXT)
            surf.blit(txt,(WIDTH-500,90))
            # Death counter
            txt=font.render(f"Deaths: {self.death_count}",True,ACCENT)
            surf.blit(txt,(WIDTH-140,18))
            # Log
            pygame.draw.rect(surf,LOG_BG,(28,220,WIDTH-56,250),border_radius=10)
            for i,line in enumerate(self.log):
                surf.blit(font.render(line,True,TEXT),(38,420 - i*24))
            # Buttons
            for b in self.ui_buttons:
                b.draw(surf)

# -------------------- Main --------------------
def main():
    game=TurnQuestGame()
    while True:
        dt=clock.tick(FPS)/1000.0
        for ev in pygame.event.get():
            if ev.type==pygame.QUIT:
                pygame.quit()
                sys.exit(0)
            game.handle_events(ev)
        game.update(dt)
        game.draw(screen)
        pygame.display.flip()

if __name__=="__main__":
    main()


SystemExit: 0